# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [ ]:
#!pip install aqua

In [2]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [3]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the slurm job

In [198]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
    job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50
#SBATCH --error=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/logs/dask-worker-%j.err
#SBATCH --output=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/output/dask-worker-%j.out

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.4:36899 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/e

### Loading the slurm job to the queue 

In [199]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [205]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4574859   compute dask-wor  b382267 PD       0:00      1 (Priority)


0

### Canceling the job if it is necessary. Set the job ID manually

In [8]:
Job_ID =   4523604
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [7]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod
import src.time_memory_estimator

### Loading the extra function, which can be used in any diagnostic

In [38]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update, data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update, data_size
        break

### Loading the tropical precipitation module

In [9]:
#dask.__version__

'2023.3.1'

In [9]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [265]:
while True:
    try:
        importlib.reload(src.time_memory_estimator)
        from  src.time_memory_estimator import time_estimator,   timestep_desirable, mem_estimator, adaptive_load, mem_units_converter, time_units_converter
        break
    except NameError and AttributeError:
        import src.time_memory_estimator
        from  src.time_memory_estimator import time_estimator,   timestep_desirable, mem_estimator, adaptive_load, mem_units_converter, time_units_converter
        break

In [11]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [12]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [13]:
import aqua

In [14]:
from aqua import Reader
from aqua.reader import catalogue

In [15]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# ICON data

In [16]:
reader = Reader(model="ICON", exp="ngc2009",  configdir=configdir, source="atm_2d_ml_R02B09", regrid="r200")
ICON_2009_reg = reader.retrieve()

In [17]:
ICON_2009_reg 

<xarray.Dataset>
Dimensions:   (time: 37009, cell: 20971520)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20 ... 2022-03-01
Dimensions without coordinates: cell
Data variables: (12/33)
    cfh_lnd   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tciw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tclw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    cptgzvi   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mslhfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msshfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    ...        ...
    10u       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    10v       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-04-02 04:02:06 retrie...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

## Time check

In [107]:
ICON_chunk = ICON_2009_reg['tprate'][0:20,:]

In [108]:
%timeit -r 2 -n 1 -o  diag.hist1d_fast(ICON_chunk,  preprocess = False)

3.28 s ± 30 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 3.28 s ± 30 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [109]:
result = _
time_fast_reg, time_fast_reg_stdev =  result.average, result.stdev 

In [110]:
time_fast_reg

3.27808556612581

In [116]:
tot_calc_time = time_estimator(ds_part = ICON_chunk, calc_time = time_fast_reg, ds_full = ICON_2009_reg['tprate'])

In [179]:
print(tot_calc_time, 's')
time_units_converter(old_value = tot_calc_time, desirable_time_unit = 'H')

6065.933435837505 s


1.6849815099548626

In [181]:
timestep_desirable(ds_part = ICON_chunk, calc_time = time_fast_reg, wanted_time = '1H', ds_full = ICON_2009_reg['tprate'])

(0.5934783225168969, 21964)

## Memory check

In [53]:
hist_fast_ICON  = diag.hist1d_fast(ICON_2009_small,  preprocess = False)

<xarray.DataArray (bin: 15)>
array([2.0955647e+09, 1.0739640e+06, 3.0778200e+05, 1.1671100e+05,
       4.9517000e+04, 2.1742000e+04, 9.9410000e+03, 4.4030000e+03,
       1.8950000e+03, 7.7700000e+02, 3.5100000e+02, 1.4100000e+02,
       5.4000000e+01, 1.5000000e+01, 3.0000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-06 1.333e-05 ... 8e-05 8.667e-05 9.333e-05
Attributes: (12/16)
    CDI_grid_type:                unstructured
    cell_methods:                 t: mean
    long_name:                    Mean total precipitation rate
    number_of_grid_in_reference:  1
    param:                        52.1.0
    standard_name:                pr
    ...                           ...
    shortName:                    tprate
    target_units:                 m s**-1
    factor:                       0.001
    offset:                       0
    src_units:                    kg m-2 s-1
    units_fixed:                  1

In [218]:
VmRSS_1, Vm_1_units = src.time_memory_estimator.read_VmRSS()
mem_units_converter(old_mem_unit =  Vm_1_units, old_value = VmRSS_1, desirable_mem_unit = 'GB')

src.time_memory_estimator.read_VmRSS_av(iter_max = 5)

VmRSS:	 2762208 kB


(2762208.0, 'kB')

In [215]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4574859   compute dask-wor  b382267  R       8:29      1 l40149


0

In [232]:
data_size(ICON_2009_reg)*8/(1024)**4

5.647125244140625

In [237]:
_ = ""

In [250]:
max_time_step=25

In [268]:

VmRSS_1, Vm_1_units = src.time_memory_estimator.read_VmRSS_av()
"""     """     """     """     """     """
reader = Reader(model="ICON", exp="ngc2009",  configdir=configdir, source="atm_2d_ml_R02B09", regrid="r200")
ICON_2009_reg = reader.retrieve()
ICON_2009_small = ICON_2009_reg['tprate'][0:max_time_step,:]
ICON_2009_small = ICON_2009_small.compute()

#ICON_2009_small = xarray_attribute_update(ICON_2009_small, ICON_2009_reg)
# Fastest histogram
hist_fast_ICON  = diag.hist1d_fast(ICON_2009_small,  preprocess = False)
"""     """     """     """     """     """
mem_max, mem_units = mem_estimator(ds_part = ICON_2009_small,  ds_full = ICON_2009_reg['tprate'], VmRSS_1 = VmRSS_1 )
mem_units_converter(old_mem_unit =  mem_units, old_value = mem_max, desirable_mem_unit = 'GB')

2.0904541015625e-06


1.5473123168945313

In [209]:
mem_units_converter(old_mem_unit =  mem_units, old_value = mem_max, desirable_mem_unit = 'GB')

2935.9192377090453

In [177]:
time_interpreter(ICON_2009_reg)

time_interpreter(ICON_2009_small) 

'30m'